In [1]:
!pip install numpy==1.19.5

import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dense

C:\Users\GamePC\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\GamePC\miniconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\GamePC\miniconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def normalize_data(rows):
    max_columns = rows.max(axis=0)
    min_columns = rows.min(axis=0)
    norm_rows = np.empty((rows.size, 52))
    for i in range(rows.shape[0]):
        for j in range(rows.shape[1]):
            if ((max_columns[j]-min_columns[j])>0):
                norm_rows[j][i] = (rows[j][i] - min_columns[j])/(max_columns[j]-min_columns[j])
            else:
                norm_rows[j][i] = 0
    return norm_rows

normalized = normalize_data(data)
normalized

NameError: name 'data' is not defined

In [2]:
def import_data():
    data = pd.read_csv('data_kia.csv').values
    x_data = data[:, :51]
    y_data = data[:, 52]
    return x_data, y_data

data, labels = import_data()
data

array([[268.8, 0.0, 5.2, ..., -8.8, 0, -3.4],
       [243.2, 0.0, 6.1, ..., -0.2, 0, -3.6],
       [217.6, 0.0, 5.2, ..., -0.2, 0, -3.6],
       ...,
       [345.6, 0.0, 6.6, ..., 0.0, 0, -13.2],
       [332.8, 0.0, 5.7, ..., 0.0, 0, -13.3],
       [281.6, 0.0, 5.2, ..., -0.1, 0, -13.3]], dtype=object)

In [3]:
def onehot_labels(labels, num_labels=10):
    """
    Converts label arrya to 1-hot encoded vectors
    """

    result = []

    for l in labels:
        label_vector = [0] * num_labels
        index = ord(l) - ord('A')
        label_vector[index] = 1

        result.append(label_vector)

    return np.array(result)

y_data = tf.convert_to_tensor(onehot_labels(labels), dtype=tf.float64)


y_data

<tf.Tensor: shape=(94380, 10), dtype=float64, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [4]:
def package_data(data, window_size=16):
    result = []
    for i in range(len(data) - window_size):
        result.append(data[i:i + window_size])

    return result
x_data = tf.convert_to_tensor(package_data(data), dtype=tf.float64)
x_data[0].shape

TensorShape([16, 51])

In [5]:
def create_model(learning_rate=1e-4, window_size=16, num_features=51):
    model = Sequential()
    model.add(LSTM(160, input_shape=(window_size, num_features), return_sequences=True ))
    model.add(LSTM(200, ))

    model.add(Dense(10, activation="sigmoid"))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="categorical_crossentropy", metrics=['categorical_crossentropy'])

    return model

In [8]:
model = create_model()
model.fit(x_data, y_data[ :-16], epochs=10, batch_size=128)

Epoch 1/10
738/738 [==============================] - 10s 13ms/step - loss: 2.0379 - categorical_crossentropy: 2.03790s - loss: 2.0567 - ca
Epoch 2/10
738/738 [==============================] - 11s 14ms/step - loss: 1.7197 - categorical_crossentropy: 1.7197
Epoch 3/10
738/738 [==============================] - 10s 14ms/step - loss: 1.5495 - categorical_crossentropy: 1.5495
Epoch 4/10
738/738 [==============================] - 10s 14ms/step - loss: 1.4017 - categorical_crossentropy: 1.4017
Epoch 5/10
738/738 [==============================] - 11s 15ms/step - loss: 1.2801 - categorical_crossentropy: 1.2801
Epoch 6/10
738/738 [==============================] - 10s 13ms/step - loss: 1.1800 - categorical_crossentropy: 1.1800
Epoch 7/10
738/738 [==============================] - 12s 16ms/step - loss: 1.0981 - categorical_crossentropy: 1.0981: 0s - loss: 1.0982 - categorical_crossent
Epoch 8/10
738/738 [==============================] - 11s 14ms/step - loss: 1.0266 - categorical_crossentropy: